In [1]:
!pip install transformers datasets accelerate evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is i

In [2]:
import pandas as pd
url = 'https://github.com/e9t/nsmc/raw/master/ratings_train.txt'
df = pd.read_csv(url, sep='\t').dropna()

In [3]:
df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [4]:
from datasets import Dataset

#df에서 한 행 씩 읽어 dict으로 반환하는 생성자
def gen():
    for row in df.itertuples():
        yield {'text': row.document, 'label': row.label}

#생성자로부터 데이터셋 만들기
ds = Dataset.from_generator(gen)

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "skt/kogpt2-base-v2",
    bos_token='</s>', #b
    eos_token='</s>', #end
    unk_token='<unk>', #모름
    pad_token='<pad>', #padding
    mask_token='<mask>'
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
#토큰화 : 64새 토큰보다 길면 자르고(truncation), 모자(pad)르면 토큰으로 채움(padding)
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length",
truncation=True, max_length=64)

tokenized_ds = ds.map(tokenize_function, batched=True)

Map:   0%|          | 0/149995 [00:00<?, ? examples/s]

In [7]:
#데이터 분할
total_size = len(ds) #최대길이

# 90%는 훈련용으로 사용
train_size = int(total_size * 0.9)
train_ds = tokenized_ds.shuffle(seed=42).select(range(train_size))
# 나머지는 검증용
eval_ds = tokenized_ds.shuffle(seed=42).select(range(train_size,
total_size))

In [8]:
import torch
torch.cuda.is_available()

True

In [9]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "skt/kogpt2-base-v2", num_labels=2)

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at skt/kogpt2-base-v2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
model.to('cuda:0');

In [11]:
import evaluate
metric = evaluate.load("accuracy")

In [12]:
import numpy as np
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1) #argmax : 누가 젤 큰지,
    return metric.compute(predictions=predictions, references=labels)


In [13]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="test_trainer",
    num_train_epochs=1,
    evaluation_strategy="epoch"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


모델 정의

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    compute_metrics=compute_metrics,
)

학습

In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.365400,0.352005,0.853867


TrainOutput(global_step=16875, training_loss=0.3943826298466435, metrics={'train_runtime': 2753.9977, 'train_samples_per_second': 49.018, 'train_steps_per_second': 6.127, 'total_flos': 4409219355770880.0, 'train_loss': 0.3943826298466435, 'epoch': 1.0})

파일 저장

In [15]:
model_path = 'drive/MyDrive/kogpt2-nsmc'
model.save_pretrained(model_path)

In [19]:
from transformers import pipeline

model.config.id2label = {0: 'NEGATIVE', 1: 'POSITIVE'}

sm = pipeline('text-classification', model=model,
              tokenizer=tokenizer, device='cuda:0')

In [20]:
#POSITIVE
sm('이 영화 재밌다')

[{'label': 'POSITIVE', 'score': 0.977456271648407}]

In [21]:
#NEGATIVE
sm('돈 아깝다')

[{'label': 'NEGATIVE', 'score': 0.9970884919166565}]

## BERT 문장 분류

In [22]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [23]:
from transformers import AutoModelForSequenceClassification

seq_model = AutoModelForSequenceClassification.from_pretrained(
    "klue/bert-base",
    num_labels=7,
    id2label={
        0: 'IT과학',
        1: '경제',
        2: '사회',
        3: '생활문화',
        4: '세계',
        5: '스포츠',
        6: '정치'
        }
    )
seq_model.to('cuda:0')

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [25]:
from datasets import load_dataset
ynat_train = load_dataset('klue', 'ynat', split='train')
ynat_val = load_dataset('klue', 'ynat', split='validation')

README.md:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/4.17M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/847k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45678 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/9107 [00:00<?, ? examples/s]

In [26]:
def convert_example(example):
    result = tokenizer(example['title'])
    result['label'] = [example['label']]
    return result

In [27]:
ynat_train_dataset = ynat_train.map(convert_example)
ynat_val_dataset = ynat_val.map(convert_example)

Map:   0%|          | 0/45678 [00:00<?, ? examples/s]

Map:   0%|          | 0/9107 [00:00<?, ? examples/s]

In [28]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="test_trainer",
    num_train_epochs=1,
    evaluation_strategy="epoch"
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [29]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [30]:
trainer = Trainer(model=seq_model,
                  args=training_args,
                  train_dataset=ynat_train_dataset,
                  eval_dataset=ynat_val_dataset,
                  data_collator=data_collator,
                  compute_metrics=compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.370900,0.388627,0.875371


TrainOutput(global_step=5710, training_loss=0.4481271934175241, metrics={'train_runtime': 588.9766, 'train_samples_per_second': 77.555, 'train_steps_per_second': 9.695, 'total_flos': 464459140498200.0, 'train_loss': 0.4481271934175241, 'epoch': 1.0})

In [31]:
model_path = 'drive/MyDrive/bert-ynat'
seq_model.save_pretrained(model_path)

In [32]:
from transformers import pipeline
text_cls = pipeline('text-classification', model=seq_model,
                    tokenizer=tokenizer, device='cuda:0')

In [33]:
text_cls('2회 연속 은메달 우상혁 "아쉽지만 파리올림픽 있어."')

[{'label': '스포츠', 'score': 0.9966602325439453}]